In [15]:
import numpy as np
import random;
from matplotlib import pyplot as plt

dataSet = np.load('kmeans_data.npy')

class kMeans:
  def __init__(self, k, dataSet):
    self.k = k
    self.dataSet = dataSet
    self.centeroids = []
    self.ClusterIds = {}
  def findInitialCenteroids(self):
    for i in range(self.k):
      while(True):
        centeroidIndex = random.randint(0,230)
        if(centeroidIndex not in self.centeroids):
          self.ClusterIds[i] = []
          self.centeroids.append(centeroidIndex)
          break
    for i in range(len(self.centeroids)):
      self.centeroids[i] = self.dataSet[self.centeroids[i]]
  def calculateDistance(self, a, b):
    return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5
  def algorithim(self):
    self.findInitialCenteroids()
    for i1 in range(1000):
      for i2 in range(len(dataSet)):
        minList = []
        for i3 in range(len(self.centeroids)):
          if i3 == 0:
            distance = self.calculateDistance(dataSet[i2], self.centeroids[i3])
            minList.append(distance)
            minList.append(i3)
          else:
            distance = self.calculateDistance(dataSet[i2], self.centeroids[i3])
            if(distance < minList[0]):
              minList[0] = distance
              minList[1] = i3
        self.ClusterIds[minList[1]].append(dataSet[i2])
      # print(ClusterIds)
      newCenteroidPoints = []  
      for i2 in range(self.k):
        meanPoint = np.mean(np.array(self.ClusterIds[i2]), axis = 0)
        newCenteroidPoints.append(meanPoint)
      #Checking for convergence 
      newCenteroidPoints = np.array(newCenteroidPoints)
      self.centeroids = np.array(self.centeroids)
      flag = True
      for i2 in range(self.k):
        if self.calculateDistance(newCenteroidPoints[i2], self.centeroids[i2]) > 10**(-6):
          self.centeroids = newCenteroidPoints
          flag = False
          break
      if(flag):
        for i2 in range(self.k):
          print("Number of points in cluster Id ", i2, " are ", len(self.ClusterIds[i2]))
        break
      else:
        # EmptyingClusters
        for i2 in range(self.k):
          self.ClusterIds[i2] = []
  def silhouetteAnalyssis(self):
    silhouetteScores = []
    for i1 in range(self.k):
      for dataPoint in self.ClusterIds[i1]:
        a = 0
        minb = 1000000
        for i2 in range(self.k):
          if (i1 == i2):
            for dataPoint2 in self.ClusterIds[i2]:
             a = a + self.calculateDistance(dataPoint, dataPoint2)
            a = a/(len(self.ClusterIds[i2])-1)
          else:
            b = 0
            for dataPoint2 in self.ClusterIds[i2]:
              b = b + self.calculateDistance(dataPoint, dataPoint2)
            b = b/(len(self.ClusterIds[i2]))
            if(b < minb):
              minb = b
        silhouetteScore = (minb-a)/max(a, minb)
        silhouetteScores.append(silhouetteScore)
    silhouetteScores = np.array(silhouetteScores)

    return np.mean(silhouetteScores)

def findOptimalK(dataSet):
  max = [-2, 0]
  for i in range(2, 7):
    tester = kMeans(i, dataSet)
    tester.algorithim()
    score = tester.silhouetteAnalyssis()
    print("For k = ", i, " silhouette score is ", score)
    if(score > max[0]):
      max[0] = score
      max[1] = i
  print("Max silhouette score is ", max[0], " at k ", max[1])
  return max[1]
def fuzzyCMeans(c, m, beta,  DataSet):
  # c = optimalK(DataSet)
  # c = 2
  # SelectingCenteroids
  centeroids = []
  ClusterIds = {}
  for i in range(c):
    while(True):
      centeroidIndex = random.randint(0, 299)
      if(centeroidIndex not in centeroids):
        ClusterIds[i] = []
        centeroids.append(centeroidIndex)
        break
  print(centeroids)
  centeroidPoints = []
  for i in centeroids:
    centeroidPoints.append(dataSet[i])
  print(centeroidPoints)
  Uprev = []
  for i99 in range(1000):
    # MembershipMatrix
    U = []
    for i1 in range(len(DataSet)):
      ithRow = []
      for i2 in range(c):
        denominator = 0
        for i3 in range(c):
          n1 = abs(DataSet[i1][0] - centeroidPoints[i2][0]) + abs(DataSet[i1][1] - centeroidPoints[i2][1])
          n2 = abs(DataSet[i1][0] - centeroidPoints[i3][0]) + abs(DataSet[i1][1] - centeroidPoints[i3][1])
          if(n1 == 0):
            n1 = 1
          if(n2 == 0):
            n2 = 1
          denominator = denominator + (n1/n2)**((2/(m - 1)))
        # if(denominator == 0):
        #   print(denominator)
        #   print(i2, i3)
        #   break
        jthEntery = 1/denominator
        ithRow.append(jthEntery)  
      U.append(ithRow)
    if(i99 != 0):
      Udifference = 0
      for i5 in range(len(DataSet)):
        for i6 in range(c):
          Udifference = Udifference + (U[i5][i6] - Uprev[i5][i6])**2
      if((Udifference)**(0.5) < beta):
        # Calculating Objective function
        J = 0
        for i5 in range(len(DataSet)):
          for i6 in range(c):
            J = J + (U[i5][i6]**m)*((DataSet[i5][0] - centeroidPoints[i6][0])**2 + (DataSet[i5][1] - centeroidPoints[i6][1])**2)
        print("Value of J is ", J)
        break


    print(U)
    # Recomputing Centeroids
    for j in range(c):
      Vjx = 0
      Vjy = 0
      denominator = 0
      for i in range(len(DataSet)):
        Vjx = Vjx + ((U[i][j])**m)*(DataSet[i][0])
        Vjy = Vjy + ((U[i][j])**m)*(DataSet[i][1])
        denominator = denominator + ((U[i][j])**m)
      centeroidPoints[j] = [Vjx/denominator, Vjy/denominator]
    Uprev = U


c = findOptimalK(dataSet)
# Optimal value calculated are 5 and 3
fuzzyCMeans(5, 2, 0.3, dataSet)



[266, 153, 161, 278, 173]
[array([4.61011124, 4.90770008]), array([-4.34906383, -4.58324669]), array([-0.75870102, -1.55271618]), array([3.66082597, 5.25447254]), array([-0.71954097, -1.46082953])]
[[0.08496370597840251, 0.0008788063192881138, 0.0021507308776190686, 0.9098071568981843, 0.0021995999265061895], [0.12405364492853206, 0.053719920460845526, 0.32750969034937905, 0.14729593562547227, 0.34742080863577113], [0.003921560776966761, 0.9365442373296644, 0.028185367570791646, 0.0041845644375355494, 0.02716426988504152], [0.19989858823669432, 0.0005916755987748203, 0.0015311914263725008, 0.7964097807605063, 0.0015687639776519574], [0.22251440000976092, 0.04929791085387586, 0.2210587197310764, 0.27595622086140636, 0.23117274854388048], [0.30663340509792153, 0.0005633598180832007, 0.0013769742996289887, 0.6900180549086186, 0.001408205875747737], [0.5427961446103192, 0.012927045622688012, 0.026645560043869927, 0.3905201087511785, 0.02711114097194418], [0.07301695625502481, 0.04740195954